In [1]:
# %load main.py
import re
import os
import csv
import json
import sys
import math
from tkinter import messagebox,Tk
from os import listdir
from decimal import Decimal
import xlrd
import datetime


root=Tk()
root.withdraw()
root.wm_attributes('-topmost',1)
date = str(datetime.date.today()).replace('-','')
#比對內容使用的物件
class compareInfo:
    def __init__(self,orderno = None, qty = 0, price = 0.0, bs = None, fw = None, sumPrice = 0.0, 
                  time= None,contract = None, broker= None):
        self.orderno = orderno
        self.qty = qty
        self.price = price
        self.bs = bs
        self.fw = fw
        self.sumPrice = sumPrice
        self.contract = contract
        self.time = time
        self.broker = broker

#商品對照檔物件
class productInfo:
    def __init__(self, Magnification = 0, Denominator = 1, nonAS400ChkDenominator = None):
        self.Magnification = Magnification
        self.Denominator = Denominator
        self.nonAS400ChkDenominator = nonAS400ChkDenominator

    def getPriceQtyInfo(self,dataPrice,dataQty,newPrice,newQty):      
        sumPrice =float(dataPrice + (newPrice * newQty))
        totalQty = int(dataQty) + int(newQty)
        if newQty > 0:
            avgPrice = float(sumPrice/totalQty)
        else:
            avgPrice = 0

        transPrice = round(avgPrice,4)
        return sumPrice,totalQty,transPrice

    def transMatchPriceforProduct(self, price, ptype, productInfo, productId): 
        if price == None or price.strip() == '':
            return None
        price = price.replace(',','').strip()
        if "'" in price:
            price = price.partition("'")
            integer = price[0]
            point = price[2]
            price = float(integer+'.'+point)
        price = float(price)
        integer = math.trunc(price)
        point = round(price - integer,4)
        data = productInfo.get(productId)
        matchprice = price
        if data != None:
            if ptype == "PATS":
                matchprice = round(matchprice * data.Magnification,4)               
                if int(data.nonAS400ChkDenominator) != 1:                                  
                     integer = round(integer * data.Magnification,4)
                     point = round(point * 100 / data.Denominator,4)
                     matchprice = integer + point
            elif ptype == "NEWTT":
                if data.nonAS400ChkDenominator != None:
                    if int(data.nonAS400ChkDenominator) == 2:
                        point = round(point* data.Magnification,4)
                        matchprice = integer + point
                    elif int(data.nonAS400ChkDenominator) == 3:
                        point = round(point* data.Magnification,3)
                        matchprice = integer + point
                    elif int(data.nonAS400ChkDenominator) == 1:
                        matchprice = round(matchprice * data.Magnification,4)
        return matchprice

#開啟檔案(檔名必須固定,若有調整請修改)
#設定檔
#------------------------------------------------------
with open('config.txt','r') as configFile: 
    configdata = json.load(configFile) 

if configdata == None:
    sys.exit("Config is None")

as400patsfilter = configdata['AS400_PATS_FILITER']
as400patsDealerlist = as400patsfilter['Dealer'].split(',')
as400patsStatuslist = as400patsfilter['Status'].split(',')

as400newttfilter = configdata['AS400_NEWTT_FILITER']
as400newttDealerlist = as400newttfilter['Dealer'].split(',')
as400newtttatuslist = as400newttfilter['Status'].split(',')

configFile.close()
#------------------------------------------------------

#商品檔
#------------------------------------------------------
patsProduct = {}
as400_patsProduct = {}
with open('PATS商品轉換.csv','rt') as fPats:
    patsfile = csv.reader(fPats)
    for index,row in enumerate(patsfile):
        if index > 0:
            pipats = productInfo()
            pias400pats = productInfo()
            patskey = row[0]
            as400patskey = row[1]
            pipats.Magnification = float(row[2])
            pipats.Denominator = float(row[3])
            pipats.nonAS400ChkDenominator = str(row[4])            
            pias400pats.Magnification = None
            pias400pats.Denominator = float(row[4])
            pias400pats.nonAS400ChkDenominator = str(row[4])   
            patsProduct.update({patskey:pipats})
            as400_patsProduct.update({as400patskey:pias400pats})
ttid = {}

newttProduct = {}
as400_newttroduct = {}            
with open('TT商品轉換.csv','rt') as fNewTT:
    newttfile = csv.reader(fNewTT)
    for index,row in enumerate(newttfile):
        if index > 0:
            pinewtt = productInfo()
            pias400newtt = productInfo()
            newttkey = row[0]
            as400newttkey = row[1]
            pinewtt.Magnification = float(row[3])           
            pinewtt.Denominator = float(row[4])            
            pias400newtt.Magnification = None
            pias400newtt.Denominator = None if str(row[5]).strip() == "#N/A" else float(row[5])
            pinewtt.nonAS400ChkDenominator = pias400newtt.Denominator
            pias400newtt.nonAS400ChkDenominator = pias400newtt.Denominator
            newttProduct.update({newttkey:pinewtt})
            as400_newttroduct.update({as400newttkey:pias400newtt})
            ttid.update({row[0]:row[1]})
pfcid= dict(zip(ttid.values(), ttid.keys()))
fPats.close()
fNewTT.close()
#------------------------------------------------------

#資料檔
#------------------------------------------------------
#as400
try:
    as400 = open('PFC2.txt', "r",encoding = 'ANSI')
    #pats
    patsfile =  [[f.partition('.')[0],open(f, "r")] for f in listdir(os.getcwd()) if 'txt' in f.partition('.')[2] and 'PFC2' not in f and 'sfa' not in f]
except Exception as e:
        messagebox.showinfo("提醒",e)
    
#newtt
#-----------------------------------
#當前執行檔所在實體路徑
dirpath = os.getcwd()
# 取得所有檔案與子目錄名稱
files = listdir(dirpath)
#尋找並開啟NEWTT檔案
newtt = None

try:
    newttfile = [[f.partition('.')[0],xlrd.open_workbook(f)] for f in listdir(os.getcwd()) if 'xlsx' in f.partition('.')[2]]

            
except Exception as e:
        messagebox.showinfo("提醒",e)
#-----------------------------------
#輸出檔案
output = open('output.csv', 'w')

#儲存檔案內容的物件(目前定義為dictionary)
sourcesAS400forPATS = {}
sourcesAS400forNEWTT = {}
sourcesAS400forManual = {}
sourcesAS400forsfa = {}
sourcesPATS = {}
sourcesNEWTT = {}
sourcesNEWTTlegal = {}
sourcessfa = {}

#Create AS400forPATS,AS400forNEWTT Data
pi = productInfo()
for line in as400:
    ci = compareInfo()
    spar = line.split()
    #AS400取得資料最小位置長度
    try:
        if '='*197 not in  spar and '-'*198 not in  spar and '\x1a' not in  spar\
            and '機密等級' not in  spar and '總合計：' not in  spar and '總筆數：' not in  spar:
            if  (spar[-2].isdigit() and len(spar[-2]) == 5) or  (spar[-1].isdigit() and len(str(spar[-1] == 5))):
                index = spar.index('N')
                #as400 pats 上手資料過濾
                orderno = spar[0].strip()
                brkordid = spar[2].strip()
                qtybid = int(spar[index+4].strip())
                qtyoffer = int(spar[index+5].strip())
                matchQty = qtybid + qtyoffer
                #轉換價格
                #--------------------------------------
                productid = spar[index+1].strip()                
                matchprice = spar[index+6].strip()
                if len(brkordid)==7:
                    matchprice = pi.transMatchPriceforProduct(matchprice,"AS400",as400_patsProduct,productid)
                else:
                    matchprice = pi.transMatchPriceforProduct(matchprice,"AS400",as400_newttroduct,productid)
        
               #--------------------------------------
                bs = "Buy" if qtybid > 0 else "Sell" if qtyoffer > 0 else None if qtybid > 0 and qtyoffer > 0 else None
                ci.orderno = orderno                        
                ci.qty = matchQty
                ci.bs = bs
                ci.broker = orderno
                ci.sumPrice = matchprice * matchQty
                ci.price = float(ci.sumPrice/matchQty)
                ci.contract = productid.strip()
                ci.time = spar[-4][:8] if len(spar[-4]) == 11 else spar[-3][:8]
                datacheck = lambda patsdata,newttdata : patsdata if patsdata != None \
                else(newttdata if newttdata != None else None)
                data = datacheck(sourcesAS400forPATS.get(brkordid),sourcesAS400forNEWTT.get(brkordid[:7]))
                if data != None:
                    sumPrice,totalQty,price = pi.getPriceQtyInfo(data.sumPrice,data.qty,matchprice,matchQty)
                    ci.sumPrice = sumPrice
                    ci.qty = totalQty
                    ci.price = float(price)
                if len(brkordid)==7: 
                    sourcesAS400forPATS.update({brkordid:ci})
                elif len(brkordid)==10 and date not in brkordid:
                    sourcesAS400forNEWTT.update({brkordid[:7]:ci})
                else:
                    data =  datacheck(sourcesAS400forManual.get(ci.contract+'-'+str(ci.price)),sourcesAS400forsfa.get(ci.contract+'-'+str(ci.price)))
                    if data != None:
                        sumPrice,totalQty,price = pi.getPriceQtyInfo(data.sumPrice,data.qty,matchprice,matchQty)
                        ci.sumPrice = sumPrice
                        ci.qty = totalQty
                        ci.price = price
                    if  date  in brkordid or spar[-5].strip() =='HKF':
                        sourcesAS400forsfa.update({ci.contract+'-'+str(ci.price).strip():ci})
                    else:
                        sourcesAS400forManual.update({ci.contract+'-'+str(ci.price).strip():ci})
    except Exception as e:
        messagebox.showinfo('提醒',str(e)+' PFC2 ERROR:'+str(spar))
as400.close()      
#Create PATS Data
try:
    for pats in patsfile:
        for line in pats[1]:
            ci = compareInfo()
            spar = line.replace('\n','').split(':')
            if spar[0] == "Order ID":
                patsbrkordid = spar[1].strip()
            elif spar[0] == "Instrument":
                contract = spar[1].strip()
                spar2 = spar[1].strip().split('/')
                if len(spar2) > 1:
                    productid = spar2[1].strip()
            elif spar[0] == "Buy or Sell":  
                patsbs = "Buy" if spar[1].strip() == "Buy" else "Sell" if spar[1].strip() == "Sell" else None  
            elif spar[0] == "2nd Reference":
                spar2 = spar[1].strip().split(',')
                if len(spar2) > 1:
                    patsorderno = spar2[1].strip()             
            elif spar[0] == "Av Fill Price": 
                if patsbrkordid == "1715275":
                   a = 1         
                patsprice = spar[1].strip()
                patsprice = pi.transMatchPriceforProduct(patsprice,"PATS",patsProduct,productid)      
            elif spar[0] == "Total Filled":  
                patsFilleqdty = spar[1].strip()
            elif spar[0] == "Working Lots":  
                patsWorkingqty = spar[1].strip()        
            elif spar[0] == "Status":  
                #as400 pats status 過濾
                patsfw = spar[1].strip() if as400patsStatuslist.__contains__(spar[1].strip()) else None
            elif 'Time Host Recd' in spar[0]:
                patstime = line.split()[4].strip()
            elif spar[0] == "Order Route" and patsfw != None:  #每一筆結束符號(依來源文件檔判斷,若非則須修改)
                if patsbrkordid == "1715275":
                   a = 1
                ci.orderno = patsorderno 
                if patsfw == "Filled":
                    patsmatchqty = patsFilleqdty
                elif patsfw == "Working":
                    patsmatchqty = patsWorkingqty
                elif patsfw == "Balance Cancelled":
                    patsmatchqty = patsFilleqdty
                else:
                    patsmatchqty = "0"
                ci.time = patstime
                ci.contract = contract
                ci.qty = int(patsmatchqty)
                ci.bs = patsbs
                ci.fw = patsfw
                ci.broker = pats[0]
                ci.sumPrice = patsprice * int(patsmatchqty)
                if int(patsmatchqty) > 0:
                    ci.price = ci.sumPrice/int(patsmatchqty)
                data = sourcesPATS.get(patsbrkordid)
                if data != None:
                    sumPrice,totalQty,price = pi.getPriceQtyInfo(data.sumPrice,data.qty,patsprice,int(patsmatchqty))
                    ci.sumPrice = sumPrice
                    ci.qty = totalQty
                    ci.price = price
                    if data.bs != patsbs:
                        ci.bs = None
                    if data.fw != patsfw:
                        ci.fw == None                
                sourcesPATS.update({patsbrkordid:ci})
    pats[1].close()
                
        
except Exception as e:
    messagebox.showinfo('提醒',e)

changePtoT = lambda bkid : pfcid.get(bkid)
changeTtoP = lambda bkid : ttid.get(bkid)
for newtt in newttfile:
    worksheet=newtt[1].sheet_by_index(0)
    nrows=worksheet.nrows
    for line in range(nrows):
        #Create NEWTT Data
        ci = compareInfo()
        brkordid = worksheet.row_values(line)[2][:7]  
        orderno = None            
        matchQty = int(worksheet.row_values(line)[8])
        productid = worksheet.row_values(line)[4]
        matchprice = worksheet.row_values(line)[9]
        fw = None
        matchprice = pi.transMatchPriceforProduct(matchprice,"NEWTT",newttProduct,productid)
        bs = "Buy" if worksheet.row_values(line)[7] == "B" else "Sell" if worksheet.row_values(line)[7] == "S" else None
        ci.orderno = orderno
        ci.qty = int(matchQty)
        ci.bs = bs
        ci.fw = fw
        ci.sumPrice = float(matchprice) * int(matchQty)
        ci.price = ci.sumPrice/int(matchQty)
        ci.contract = productid.strip()
        ci.time = worksheet.row_values(line)[11][:8]
        ci.broker = newtt[0]
        if (len(worksheet.row_values(line)[12]) == 0 and '*' not in worksheet.row_values(line)[3]) or len(str(worksheet.row_values(line)[6])) != 0:
            try:
                data = sourcesNEWTTlegal.get(changeTtoP(ci.contract)+'-'+str(ci.price))
            except TypeError:
                messagebox.showinfo('提醒','請新增品檔 :'+ci.contract)
            if data != None:
                sumPrice,totalQty,price = pi.getPriceQtyInfo(data.sumPrice,data.qty,matchprice,int(matchQty))
                ci.sumPrice = sumPrice
                ci.qty = totalQty
                ci.price = price
            try:        
                sourcesNEWTTlegal.update({changeTtoP(ci.contract)+'-'+str(ci.price):ci})
            except:
                messagebox.showinfo('提醒',f'請新增品檔{ci.contract}')
       
        else:
            if (len(worksheet.row_values(line)[12])) != 0: 
                data = sourcesNEWTT.get(brkordid)
                if data != None:
                    sumPrice,totalQty,price = pi.getPriceQtyInfo(data.sumPrice,data.qty,matchprice,int(matchQty))
                    ci.sumPrice = sumPrice
                    ci.qty = totalQty
                    ci.price = price
                    if data.bs != bs:
                        ci.bs = None
                    if data.fw != fw:
                        ci.fw == None
                sourcesNEWTT.update({brkordid:ci})
    del newtt
    
    
#Create sfa Data
date = str(datetime.date.today()).replace('-','')
with open('sfa.txt','rt',encoding = 'UTF-8') as sfa:
    for line in sfa:
        nline = line.replace('\n','').split()
        ci = compareInfo()
        if date in nline[13][2:10]:
            matchQty = nline[6].strip()
            matchprice = float(nline[4].strip())
            ci.orderno = nline[13].strip()
            ci.qty = matchQty
            ci.price = matchprice
            ci.contract = nline[1].strip()
            ci.time = nline[9].strip()
            ci.broker = 'sfa'
            ci.sumPrice = float(matchprice) * int(matchQty)
            data = sourcessfa.get(ci.contract+'-'+str(ci.price))
            if data != None:
                sumPrice,totalQty,price = pi.getPriceQtyInfo(data.sumPrice,data.qty,matchprice,int(matchQty))
                ci.sumPrice = sumPrice
                ci.qty = totalQty
                ci.price = price
            sourcessfa.update({ci.contract+'-'+str(ci.price).strip():ci})    
#------------------------------------------------------

output.write(f"target VS compare,item,brkordid,orderno,status,target value,self value,contract,time\n")


# PATS => PFC2 pats error
#===============================================================================
for key,value in sourcesPATS.items():
    data = sourcesAS400forPATS.get(key)
    if key == "1761855":
        a = 1
    if data == None:  
        if value.fw !='Working':
            output.write(f"pats error,pfc2 not exist,{key},  ,  , , ,{value.contract} ,{value.time},{value.broker}\n")
    elif data.qty != value.qty:
        output.write(f"")
        output.write(f"pats error,match qty err,{key},{data.orderno}, ,{data.qty},{value.qty},{value.time},{value.broker}\n")
    elif data.bs != value.bs:
        output.write(f"pats error,buy sell err,{key},{data.orderno}, ,{data.bs},{value.bs},{value.time},{value.broker}\n")
    elif round(data.price,4) != round(value.price,4):
        output.write(f"pats error,price err,{key},{data.orderno},  ,{round(data.price,4)},{round(value.price,4)}, ,{value.time},{value.broker}\n") 
    elif data.orderno != value.orderno:
        output.write(f"pats error,orderno err,{key},,{data.orderno}, ,{value.orderno},{value.time},{value.broker}\n")
# PFC2 => PATS pfc2 error
#===============================================================================
for key,value in sourcesAS400forPATS.items():
    data = sourcesPATS.get(key)
    if data == None:
        output.write(f"pfc2pats error,pats or newtt not exist,{key}, {value.bs},{value.qty} , ,{value.price} ,{value.contract},{value.time},{value.broker}\n")
    elif data.qty != value.qty:
        output.write(f"pfc2pats error,match qty err,{key},{data.orderno}, ,{data.qty},{value.qty},{value.time},{value.broker}\n")
    elif data.bs != value.bs:
        output.write(f"pfc2pats error,buy sell err,{key},{data.orderno}, ,{data.bs},{value.bs},{value.time},{value.broker}\n")
    elif round(data.price,4) != round(value.price,4):
        output.write(f"pfc2pats error,price err,{key},{data.orderno}, ,{round(data.price,4)},{round(value.price,4)},{value.contract},{value.time},{value.broker}\n")
    elif data.orderno != value.orderno:
        output.write(f"pfc2pats error,orderno err,{key},,{data.orderno}, ,{value.orderno}\n")  
# NEWTT => PFC2 newtt error
#===============================================================================       
for key,value in sourcesNEWTT.items():
    data = sourcesAS400forNEWTT.get(key)
    if data == None:
        a = 0
        for key1,value1 in sourcesAS400forManual.items():
            if value.time == value1.time and value.price == value1.price :
                a = a+1
        if a == 0:
            output.write(f"newtt error,pfc2 not exist,{key}, ,  , ,  ,{value.contract},{value.time}, {value.broker}\n")
    elif data.qty != value.qty:
        output.write(f"newtt error,match qty err,{key},{data.orderno},{data.contract}/{value.contract},{data.qty},{value.qty}, {value.broker}\n")
    elif data.bs != value.bs:
        output.write(f"newtt error,buy sell err,{key},{data.orderno},{data.contract}/{value.contract},{data.bs},{value.bs}, {value.broker}\n")
    elif str(data.price) != str(value.price):
        output.write(f"newtt error,price err,{key},{data.orderno}, ,{'%.4f' % data.price},{'%.4f' % value.price}, {data.contract}/{value.contract},{value.time} , {value.broker}\n")
# PFC2 => NEWTT pfc2 error
#=============================================================================== 
for key,value in sourcesAS400forNEWTT.items():
    data = sourcesNEWTT.get(key)
    if data == None:
        output.write(f"pfc2newtt error, newtt not exist,{key},{value.bs} ,{value.qty} , ,{value.price} ,{value.contract},{value.time},{value.broker}\n")
    elif data.qty != value.qty:
        output.write(f"pfc2newtt error,match qty err,{key},{data.orderno}, ,{data.bs},{value.bs},{value.time},{value.broker}\n")
    elif data.bs != value.bs:
        output.write(f"pfc2newtt error,buy sell err,{key},{data.orderno}, ,{data.bs},{value.bs},{value.time},{value.broker}\n")
    elif round(data.price,4) != round(value.price,4):
        output.write(f"pfc2newtt error,price err,{key},{data.orderno}, ,{round(data.price,4)},{round(value.price,4)},{value.contract},{value.time},{value.broker}\n")
# PFC2(no brokerid) => NEWTT Manual error 法人+人工+OP
#=============================================================================== 

for key,value in sourcesAS400forManual.items():
    data = datacheck(sourcesNEWTTlegal.get(key),sourcessfa.get(key))
    if data == None:
        output.write(f"Manual error, newtt not exist,{key},{value.bs} ,{value.qty} , ,{value.price} ,{value.contract},{value.time},{value.broker}\n")
    elif data.qty != value.qty:
        output.write(f"Manual error,match qty err,{key},{data.orderno}, ,{data.qty},{value.qty}, ,{value.time},{value.broker}\n")
    elif round(data.price,4) != round(value.price,4):
        output.write(f"Manual error,price err,{key},{data.orderno}, ,{round(data.price,4)},{round(value.price,4)},{value.contract},{value.time},{value.broker}\n")

# newtt(no brokerid) => pfc2 NEWTTlegal  error 法人+人工+OP
#=============================================================================== 

for key,value in sourcesNEWTTlegal.items():
    data = sourcesAS400forManual.get(key)
    if data == None:
        output.write(f"NEWTTlegal error, pfc2 not exist,{key},{value.bs} ,{value.qty} , ,{value.price} ,{value.contract},{value.time},{value.broker}\n")
    elif data.qty != value.qty:
        output.write(f"NEWTTlegal error,match qty err,{key},{data.orderno}, ,{data.qty},{value.qty}, , {value.time},{value.broker}\n")
    elif round(data.price,4) != round(value.price,4):
        output.write(f"NEWTTlegal error,price err,{key},{data.orderno}, ,{round(data.price,4)},{round(value.price,4)},{value.contract},{value.time},{value.broker}\n")

# PFC2 => sfa pfc2sfa error 
#===============================================================================         
        
for key,value in sourcesAS400forsfa.items():        
    data = sourcessfa.get(key)
    if data == None:
        output.write(f"pfc2sfa error, sfa not exist,{key},{value.bs} ,{value.qty} , ,{value.price} ,{value.contract},{value.time},{value.broker}\n")
    elif str(data.qty) != str(value.qty):
        output.write(f"pfc2sfa error,match qty err,{key},{data.orderno}, ,{data.qty},{value.qty}, ,{value.time},{value.broker}\n")
    elif round(data.price,4) != round(value.price,4):
        output.write(f"pfc2sfa error,price err,{key},{data.orderno}, ,{round(data.price,4)},{round(value.price,4)},{value.contract},{value.time},{value.broker}\n")
#  sfa =>PFC2 sfa error 
#===============================================================================
for key,value in sourcessfa.items():        
    data = datacheck(sourcesAS400forsfa.get(key),sourcesAS400forManual.get(key))
    if data == None:
        output.write(f"sfa error, pfc2 not exist,{key},{value.bs} ,{value.qty} , ,{value.price} ,{value.contract},{value.time},{value.broker}\n")
    elif str(data.qty) != str(value.qty):
        output.write(f"sfa error,match qty err,{key},{data.orderno}, ,{data.qty},{value.qty}, ,{value.time},{value.broker}\n")
    elif round(data.price,4) != round(value.price,4):
        output.write(f"sfa error,price err,{key},{data.orderno}, ,{round(data.price,4)},{round(value.price,4)},{value.contract},{value.time},{value.broker}\n")
messagebox.showinfo('提醒','比對完成')                                          
output.close()

